In [2]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
import seaborn as sns
import xlrd
import cv2
import os
import csv
import re
import pandas as pd
import array as arr
import numpy as np
from datetime import datetime, timedelta

pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)

#Read csv files and convert it to excel files
read_file = pd.read_csv (r'apollobeddevicecetail.csv')
read_file.to_excel (r'file1.xlsx', index = None, header=True)

read_file = pd.read_csv (r'apollorefinicubbox_17april.csv')
read_file.to_excel (r'file2.xlsx', index = None, header=True)

#Open Excel file
file1 = pd.read_excel('file1.xlsx')
file2 = pd.read_excel('file2.xlsx')
file3 = pd.read_excel('MotinagarWithISt.xlsx')

In [3]:
from datetime import datetime
from dateutil import tz
from datetime import timedelta  

In [3]:
#txt_1 = pd.read_csv('finalFile_1.txt', sep=" ", header=None)

In [4]:
#txt_2 = pd.read_csv('finalFile_2.txt', sep=" ",header=None,skiprows=True)

In [5]:
#txt = pd.concat([txt_1,txt_2])

In [4]:
txt = pd.read_csv('finalFile_3.txt', sep=" ",header=None)

In [ ]:
t = pd.DataFrame(columns=['Video (Title)','START'])

In [5]:
txt = txt.rename(columns={0: 'Video (Title)', 1: 'Start'})

In [6]:
txt.dropna(inplace=True)

In [7]:
def extract_time(x):
    
    
    t = str(x).split("_")[1]
    if t != '720p':
        from_zone = tz.tzutc()
        to_zone = tz.tzlocal()
        test = t.split("-")[0] +"-" + t.split("-")[1] + "-" + t.split("-")[2] + " " + ((t.split("-")[3]).split(".")[0] +":"+ (t.split("-")[3]).split(".")[1] + ":"+ (t.split("-")[3]).split(".")[2])
        utc = datetime.strptime(test, '%Y-%m-%d %H:%M:%S')
        central = utc.astimezone(to_zone)
        central = pd.to_datetime(central)+timedelta(seconds=19800)
    else:
        t = str(x).split("_")[2]
        from_zone = tz.tzutc()
        to_zone = tz.tzlocal()
        test = t.split("-")[0] +"-" + t.split("-")[1] + "-" + t.split("-")[2] + " " + ((t.split("-")[3]).split(".")[0] +":"+ (t.split("-")[3]).split(".")[1] + ":"+ (t.split("-")[3]).split(".")[2])
        utc = datetime.strptime(test, '%Y-%m-%d %H:%M:%S')
        central = utc.astimezone(to_zone)
        central = pd.to_datetime(central)+timedelta(seconds=19800)
        
    return ((str(pd.to_datetime(central)).split("+")[0]))

def extract_neo(x):
    s = str(x).split("_")[0]
    
    return (s.split(" ")[-1].split(".")[0].lower())

def extract_seq(x):
    s = str(x).split("_")[-1]
    
    return s.split(".")[0]

def to_split_0(x):
    x = str(x)
    return x.split(".")[0]
def to_split_1(x):
    try:
        x = str(x)
        return x.split(".")[1]
    except:
        pass

def date_extract(x):
    return x.split(" ")[0]

def time_join(x,y):
    try:
    
        to_zone = tz.tzlocal()
        test = str(x)+" "+str(y)
        utc = datetime.strptime(test, '%Y-%m-%d %H:%M:%S')
        central = utc.astimezone(to_zone)
        central = pd.to_datetime(central)+timedelta(seconds=19800)
        return central
    except:
        to_zone = tz.tzlocal()
        test = str(x)+" "+str(y)
        utc = datetime.strptime(test, '%Y-%m-%d %H.%M.%S')
        central = utc.astimezone(to_zone)
        central = pd.to_datetime(central)+timedelta(seconds=19800)
        return central
    
    
        
    
    
    
    

In [8]:
file3['NEO'] = file3['Video (Title)'].apply(extract_neo)

In [9]:
file3['Start_time_NEO'] = file3['Video (Title)'].apply(extract_time)

In [10]:
t = pd.merge(file3,txt,on='Video (Title)',how='left')

In [11]:
t.dropna(subset=['Start'],inplace=True)

In [12]:
t.Start_time_NEO.iloc[0]

'2019-12-20 10:42:53'

In [13]:
t['start_date'] = t['Start_time_NEO'].apply(date_extract)

In [14]:
t['NEO_START_TIME'] = t.apply(lambda x: time_join(x['start_date'], x['Start']), axis=1)

In [15]:
file3 = t.copy()

In [16]:
file3['Tag'] = file3['Video (Title)'].apply(extract_seq)

In [17]:
file3['to_min'] = file3['To (min.sec)'].apply(to_split_1)
file3['From_min'] = file3['From (min.sec)'].apply(to_split_1)
file3['to_Hour'] = file3['To (min.sec)'].apply(to_split_0)
file3['From_Hour'] = file3['From (min.sec)'].apply(to_split_0)
file3['to_min'] = file3['to_min'].astype(float)
file3['From_min'] = file3['From_min'].astype(float)

In [18]:
def replace_NIL(x):
    if x == 'NIL' or x =='camera angle changed':
        return '0'
    else:
        return x

In [19]:
file3['to_Hour'] = file3['to_Hour'].apply(replace_NIL)
file3['to_Hour'] = file3['to_Hour'].astype(float)

In [20]:
file3['From_Hour'] = file3['From_Hour'].apply(replace_NIL)
file3['From_Hour'] = file3['From_Hour'].astype(float)

In [21]:
df = pd.DataFrame()

In [22]:
for i in file3['NEO'].unique():
    print(1)
    x = file3[file3['NEO']==i]
    print(2)
    for j in x['Video (Title)'].unique():
        #print(j)
        try:
            print(j)
            y = x[x['Video (Title)']==j]
            #for k in x['tags'].unique():
            #z = x[x['tags']==k]
            y['Duration'] = (y['to_Hour']*60 + y['to_min']) - (y['From_Hour']*60 + y['From_min'])
            #y['Duration_shift'] = y['Duration'].shift(1,axis=0)
            y['cummulative_duration'] = y['Duration'].cumsum()
            df = df.append(y,ignore_index=True)
            
            
            print(i,j,np.nansum(y['Duration']))
        
        except:
            continue

1
2
NEO109.stream_2019-12-20-05.12.53.790-UTC_1.mp4
neo109 NEO109.stream_2019-12-20-05.12.53.790-UTC_1.mp4 336.0
NEO109.stream_2019-12-23-12.41.36.889-UTC_0.mp4


C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


neo109 NEO109.stream_2019-12-23-12.41.36.889-UTC_0.mp4 244.0
NEO109.stream_2019-12-23-12.41.36.889-UTC_1.mp4
neo109 NEO109.stream_2019-12-23-12.41.36.889-UTC_1.mp4 19.0
NEO109.stream_2019-12-23-12.41.36.889-UTC_2.mp4
neo109 NEO109.stream_2019-12-23-12.41.36.889-UTC_2.mp4 68.0
NEO109.stream_2019-12-23-12.41.36.889-UTC_3.mp4
neo109 NEO109.stream_2019-12-23-12.41.36.889-UTC_3.mp4 1491.0
NEO109.stream_2019-12-23-12.41.36.889-UTC_4.mp4
neo109 NEO109.stream_2019-12-23-12.41.36.889-UTC_4.mp4 613.0
NEO109.stream_2019-12-23-12.41.36.889-UTC_5.mp4
neo109 NEO109.stream_2019-12-23-12.41.36.889-UTC_5.mp4 0.0
NEO109.stream_2019-12-23-12.41.36.889-UTC_6.mp4
neo109 NEO109.stream_2019-12-23-12.41.36.889-UTC_6.mp4 161.0
NEO109.stream_2019-12-23-12.41.36.889-UTC_7.mp4
neo109 NEO109.stream_2019-12-23-12.41.36.889-UTC_7.mp4 2.0
NEO109.stream_2019-12-23-12.41.36.889-UTC_8.mp4
neo109 NEO109.stream_2019-12-23-12.41.36.889-UTC_8.mp4 5.0
NEO109.stream_2019-12-23-12.41.36.889-UTC_9.mp4
neo109 NEO109.stream_2019-

neotiny104 NEOTINY104.stream_720p_2020-02-26-18.52.24.298-UTC_0.mp4 0.0
NEOTINY104.stream_720p_2020-02-26-18.54.54.972-UTC_0.mp4
neotiny104 NEOTINY104.stream_720p_2020-02-26-18.54.54.972-UTC_0.mp4 0.0
NEOTINY104.stream_720p_2020-02-26-18.57.00.065-UTC_0.mp4
neotiny104 NEOTINY104.stream_720p_2020-02-26-18.57.00.065-UTC_0.mp4 0.0
NEOTINY104.stream_720p_2020-02-26-21.23.20.317-UTC_0.mp4
neotiny104 NEOTINY104.stream_720p_2020-02-26-21.23.20.317-UTC_0.mp4 0.0
NEOTINY104.stream_720p_2020-02-26-21.26.16.298-UTC_0.mp4
neotiny104 NEOTINY104.stream_720p_2020-02-26-21.26.16.298-UTC_0.mp4 0.0
NEOTINY104.stream_720p_2020-02-26-21.31.29.705-UTC_2.mp4
neotiny104 NEOTINY104.stream_720p_2020-02-26-21.31.29.705-UTC_2.mp4 0.0
NEOTINY104.stream_720p_2020-02-26-21.31.29.705-UTC_3.mp4
neotiny104 NEOTINY104.stream_720p_2020-02-26-21.31.29.705-UTC_3.mp4 81.0
NEOTINY104.stream_720p_2020-02-26-21.31.29.705-UTC_4.mp4
neotiny104 NEOTINY104.stream_720p_2020-02-26-21.31.29.705-UTC_4.mp4 0.0
NEOTINY104.stream_720p_2

In [23]:
df['END'] = 0
df['cummulative_duration'].fillna(0,inplace=True)

In [24]:
for i in range(len(df)):
    df['END'].iloc[i] = pd.to_datetime(df['NEO_START_TIME'].iloc[i]) + timedelta(seconds=df['cummulative_duration'].iloc[i])

C:\Users\Dell\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [25]:
df['START'] = 0
df['Duration'].fillna(0,inplace=True)

In [26]:
for i in range(len(df)):
    df['START'].iloc[i] = pd.to_datetime(df['END'].iloc[i]) - timedelta(seconds=df['Duration'].iloc[i])

In [27]:
df.to_csv('times_apollo_5.csv')

In [56]:
g = df[df['Video (Title)']=='NEO109.stream_2019-12-23-12.41.36.889-UTC_0.mp4']

In [57]:
g

,UHID,Video (Title),Total Duration of video file (min.sec),From (min.sec),To (min.sec),Procedure,Label,Duration (seconds),Procedure_IST_Time,From_IST_Time,NEO,Start_time_NEO,Start,start_date,NEO_START_TIME,Tag,to_min,From_min,to_Hour,From_Hour,Duration,cummulative_duration,END,START
9,RSHI.0000021833,NEO109.stream_2019-12-23-12.41.36.889-UTC_0.mp4,17.49,00.00,01.54,Feeding through OG tube,Feeding,114,2019-12-23 18:11:36,2019-12-23 18:11:36,neo109,2019-12-23 18:11:36,12.41.36,2019-12-23,2019-12-23 18:11:36+05:30,0,54.0,0.0,1.0,0.0,114.0,114.0,2019-12-23 18:13:30+05:30,2019-12-23 18:11:36+05:30
10,RSHI.0000021833,NEO109.stream_2019-12-23-12.41.36.889-UTC_0.mp4,17.49,00.00,01.54,Feeding through OG tube,Feeding,114,2019-12-23 18:11:36,2019-12-23 18:11:36,neo109,2019-12-23 18:11:36,12.41.36,2019-12-23,2019-12-23 18:11:36+05:30,0,54.0,0.0,1.0,0.0,114.0,228.0,2019-12-23 18:15:24+05:30,2019-12-23 18:13:30+05:30
11,RSHI.0000021833,NEO109.stream_2019-12-23-12.41.36.889-UTC_0.mp4,17.49,12.23,12.31,Administration through IV line,Other,7,2019-12-23 18:23:59,2019-12-23 18:11:36,neo109,2019-12-23 18:11:36,12.41.36,2019-12-23,2019-12-23 18:11:36+05:30,0,31.0,23.0,12.0,12.0,8.0,236.0,2019-12-23 18:15:32+05:30,2019-12-23 18:15:24+05:30
12,RSHI.0000021833,NEO109.stream_2019-12-23-12.41.36.889-UTC_0.mp4,17.49,12.23,12.31,Administration through IV line,Other,7,2019-12-23 18:23:59,2019-12-23 18:11:36,neo109,2019-12-23 18:11:36,12.41.36,2019-12-23,2019-12-23 18:11:36+05:30,0,31.0,23.0,12.0,12.0,8.0,244.0,2019-12-23 18:15:40+05:30,2019-12-23 18:15:32+05:30


In [ ]:
time_join_test(g['start_date'].iloc[0],g['Start'].iloc[0])

In [ ]:
df

In [ ]:
def time_join_test(x,y):
    #try:
    
    to_zone = tz.tzlocal()
    test = str(x)+" "+str(y)
    utc = datetime.strptime(test, '%Y-%m-%d %H.%M.%S')
    central = utc.astimezone(to_zone)
    central = pd.to_datetime(central)+timedelta(seconds=19800)
    return central

In [42]:
print(t.iloc[477])

UHID                                                                                    NaN
Video (Title)                             NEOTINY104.stream_720p_2020-03-31-06.45.48.422...
Total Duration of video file (min.sec)                                                14.07
From (min.sec)                                                                         9.44
To (min.sec)                                                                           9.51
Procedure                                              Position change from prone to supine
Label                                                                                     -
Duration (seconds)                                                                      NaN
Procedure_IST_Time                                                      2020-03-31 12:25:32
From_IST_Time                                                           2020-03-31 12:15:48
NEO                                                                             

In [25]:
t.to_csv('check.csv')